In [11]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib
from scipy import ndimage
from scipy.ndimage import rotate, zoom
import zipfile
import random
from keras.utils import get_file
from sklearn.model_selection import train_test_split

In [15]:
url_1 = "https://github.com/hasibzunair/3D-image-classification-tutorial/releases/download/v0.2/CT-0.zip"
url_2 = "https://github.com/hasibzunair/3D-image-classification-tutorial/releases/download/v0.2/CT-23.zip"

# Directory where the files will be stored
cache_dir = os.getcwd()  # Current working directory

# Download the first file
filename_1 = get_file("CT-0.zip", url_1, cache_dir=cache_dir)

# Download the second file
filename_2 = get_file("CT-23.zip", url_2, cache_dir=cache_dir)

# Make a directory to store the data
os.makedirs("MosMedData", exist_ok=True)  # Using exist_ok=True to avoid errors if the directory exists

# Unzip the first file
with zipfile.ZipFile(filename_1, "r") as z_fp:
    z_fp.extractall("./MosMedData/")

# Unzip the second file
with zipfile.ZipFile(filename_2, "r") as z_fp:
    z_fp.extractall("./MosMedData/")

1065471431/1065471431 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step


In [16]:
for dirpath, dirnames, filenames in os.walk("MosMedData"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in 'MosMedData'.
There are 0 directories and 100 images in 'MosMedData/CT-0'.
There are 0 directories and 100 images in 'MosMedData/CT-23'.


In [14]:
# import shutil

# # Path to the directory to be deleted
# directory_path_ct23 = '/content/MosMedData/CT-23'
# directory_path_ct0 = '/content/MosMedData/CT-0'

# # Delete the directory and all its contents
# shutil.rmtree(directory_path_ct23)
# shutil.rmtree(directory_path_ct0)

In [ ]:
# Function to load a .nii.gz file
def load_nii(file_path):
    return nib.load(file_path).get_fdata()

def save_nii(volume, save_path):
    new_img = nib.Nifti1Image(volume, np.eye(4))  # Assuming the affine matrix is identity
    nib.save(new_img, save_path)

# Function to apply a random rotation to the volume
def random_rotate(volume):
    angle = random.uniform(0, 360)
    return rotate(volume, angle, axes=(0, 1), reshape=False)  # Rotate around the first two axes

# Function to apply random zoom to the volume
def random_zoom(volume):
    zoom_factor = random.uniform(0.8, 1.2)  # Random zoom factor between 0.8x and 1.2x
    return zoom(volume, zoom_factor)

def create_multiview_dataset(base_path, ct_folders):
    for ct_folder in ct_folders:
        folder_path = os.path.join(base_path, ct_folder)
        # List all items in the folder and process only files (not directories)
        for item in os.listdir(folder_path):
            item_path = os.path.join(folder_path, item)

            # Skip hidden files (those starting with a dot) and directories
            if item.startswith('.') or os.path.isdir(item_path):
                continue  # Skip hidden files or directories

            if os.path.isfile(item_path) and item.endswith('.nii.gz'):  # Check if it's a .nii.gz file
                try:
                    # Load the original CT scan image
                    original_volume = load_nii(item_path)

                    # Create subdirectory for each file (based on the file name without extension)
                    study_dir_name = os.path.splitext(item)[0]  # Get the file name without extension
                    study_save_dir = os.path.join(folder_path, study_dir_name)
                    os.makedirs(study_save_dir, exist_ok=True)

                    # Save the original image
                    save_nii(original_volume, os.path.join(study_save_dir, f"{study_dir_name}_original.nii.gz"))

                    # Save the rotated image
                    rotated_volume = random_rotate(original_volume)
                    save_nii(rotated_volume, os.path.join(study_save_dir, f"{study_dir_name}_rotated.nii.gz"))

                    # Save the zoomed-in image
                    zoomed_volume = random_zoom(original_volume)
                    save_nii(zoomed_volume, os.path.join(study_save_dir, f"{study_dir_name}_zoom.nii.gz"))

                except Exception as e:
                    print(f"Error processing {item_path}: {e}")

# Directory structure
base_directory = "MosMedData"
ct_folders = ["CT-0", "CT-23"]

# Create multi-view dataset
create_multiview_dataset(base_directory, ct_folders)


In [21]:
def count_directories(path):
    # List all items in the directory and filter for directories
    return len([d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))])

# Example usage:
path = "MosMedData/CT-23"
path2 = "MosMedData/CT-0"
print(f"Number of directories: {count_directories(path2)}")
print(f"Number of directories: {count_directories(path)}")

Number of directories: 100
Number of directories: 100


In [24]:
base_path = '/content/MosMedData/CT-23'

# Iterate through the files in MosMedData
for item in os.listdir(base_path):
    item_path = os.path.join(base_path, item)

    # Check if it's a file and if it ends with '.gz'
    if os.path.isfile(item_path) and item.endswith('.gz'):
        os.remove(item_path)  # Delete the file
        print(f"Deleted file: {item_path}")

Deleted file: /content/MosMedData/CT-23/study_1071.nii.gz
Deleted file: /content/MosMedData/CT-23/study_0989.nii.gz
Deleted file: /content/MosMedData/CT-23/study_0965.nii.gz
Deleted file: /content/MosMedData/CT-23/study_0941.nii.gz
Deleted file: /content/MosMedData/CT-23/study_1108.nii.gz
Deleted file: /content/MosMedData/CT-23/study_0967.nii.gz
Deleted file: /content/MosMedData/CT-23/study_0953.nii.gz
Deleted file: /content/MosMedData/CT-23/study_0985.nii.gz
Deleted file: /content/MosMedData/CT-23/study_0966.nii.gz
Deleted file: /content/MosMedData/CT-23/study_0958.nii.gz
Deleted file: /content/MosMedData/CT-23/study_1090.nii.gz
Deleted file: /content/MosMedData/CT-23/study_0990.nii.gz
Deleted file: /content/MosMedData/CT-23/study_1089.nii.gz
Deleted file: /content/MosMedData/CT-23/study_0975.nii.gz
Deleted file: /content/MosMedData/CT-23/study_1081.nii.gz
Deleted file: /content/MosMedData/CT-23/study_1067.nii.gz
Deleted file: /content/MosMedData/CT-23/study_1076.nii.gz
Deleted file: 

In [25]:
print(f"Number of directories: {count_directories(path2)}")
print(f"Number of directories: {count_directories(path)}")

Number of directories: 100
Number of directories: 100
